In [2]:
import requests
import pandas as pd
from lxml import etree
import re
import gevent
from gevent import monkey

In [6]:
monkey.patch_all()

In [7]:
def getHtml(url):
    headers = {
    'Cookie': 'co=764; __utma=164819386.1948347344.1561496847.1561496847.1561496847.1; __utmc=164819386; __utmz=164819386.1561496847.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __hstc=80211592.b3bc35c0093cdd56c74cb9e36cdf8673.1561496847282.1561496847282.1561496847282.1; hubspotutk=b3bc35c0093cdd56c74cb9e36cdf8673; __hssrc=1; __utmt=1; projectsPerPage=21; __atuvc=47%7C26; __atuvs=5d128d20b8ca4b8e02e; __utmb=164819386.48.10.1561496847; __hssc=80211592.48.1561496847282',
    'Host': 'www.turnerconstruction.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36'
    }
    response = requests.get(url, headers = headers)
    response.encoding = response.apparent_encoding
    return response

d:\language\python\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [16]:
def getMaxPage(html, url):
    page_str = 'pagination-list'
    if html.find(page_str) == -1:
        max_page = 1
    else:
        obj = etree.HTML(html)
        reg_page = '//div[@class="pagination-list"]/ul/li[last()- 1]/a/text()'
        max_page = int(obj.xpath(reg_page)[0])
    
    list_urls = [url + '/%s' % str(j) for j in range(1, max_page + 1)]
    return list_urls

In [54]:
def get_links(html, item):
    link_reg = '//ul[@class="featured-projects-list"]/li/p[1]/a/@href'
    name_reg = '//ul[@class="featured-projects-list"]/li/p[2]/a/text()'
    location_reg = '//ul[@class="featured-projects-list"]/li/p[3]/text()'
    
    obj = etree.HTML(html)
    links = obj.xpath(link_reg)
    names = obj.xpath(name_reg)
    locations = obj.xpath(location_reg)
    
    result = []
    for link, name, location in zip(links, names, locations):
        inner_link = 'http://www.turnerconstruction.com' + link
        info_html = getHtml(inner_link).text
        Infos = get_infos(info_html)
        info_dict = {}
        info_dict['Name'] = name
        info_dict['Location'] = location
        info_dict['Link'] = 'http://www.turnerconstruction.com' +  link
        info_dict['Info'] = Infos[0]
        info_dict['Description'] = Infos[1]
        info_dict['Type'] = item
        
        result.append(info_dict)
        
    return result

In [37]:
def get_infos(html):
    inner_obj = etree.HTML(html)
    
    feature_reg = '//div[@class="project-description-div"]/ul/li'
    
    features = inner_obj.xpath(feature_reg)
    
    clients = []
    for each in features:
        
        key = each.xpath('./span/text()')
        
        key = key[0].strip()
        
        value = each.xpath('./strong/text()')
        if len(value) == 0:
            value = each.xpath('./a/text()')
            
            if len(value) == 0:
                value = ' '
            else:
                value = ', '.join([single.strip() for single in value])
        else:
            value = value[0].strip()
        
        clients.append(key + ': ' + value)
        
    clients = '\n'.join(clients)
    
    paragraph_reg  = '//div[@id="content-side"]/p/text()'
    
    paragraphs = inner_obj.xpath(paragraph_reg)
    
    if len(paragraphs) == 0:
        paragraphs = None
    else:
        paragraphs = '\n'.join([single.strip() for single in paragraphs])
        
    return [clients, paragraphs]
    

In [42]:
def create_url(item,lists):
    base_url = 'http://www.turnerconstruction.com/experience/projects/'
    item_url = base_url + item + '/north-america'
    
    res = getHtml(item_url).text
    pages = getMaxPage(res, item_url)
    for page in pages:
        page_html = getHtml(page).text
        lists.extend(get_links(page_html, item))


d:\language\python\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [55]:
item_lists = ['commercial','education','government','healthcare','research-development','residential-hotel']
gens = []
final_list = []
for single_item in item_lists:
    gens.append(gevent.spawn(create_url, single_item, final_list))

gevent.joinall(gens)

[<Greenlet at 0x2be89cbd048: _run>,
 <Greenlet at 0x2be837d5e48: _run>,
 <Greenlet at 0x2be89cbd148: _run>,
 <Greenlet at 0x2be837d5d48: _run>,
 <Greenlet at 0x2be837d5c48: _run>,
 <Greenlet at 0x2be837d5a48: _run>]

In [56]:
print(final_list)

[{'Name': 'BWXT Y-12 Production and Public Interface Facilities', 'Location': 'Oak Ridge, TN', 'Link': 'http://www.turnerconstruction.com/experience/project/1461/bwxt-y-12-production-and-public-interface-facilities', 'Info': 'Client: Lawler-Wood Group\nLocation: Oak Ridge,  TN\nAttributes: Federal, Government, LEED Certified, Green Building, Commercial, Turner Logistics: Procurement Services, Design+Build, Tennessee', 'Description': 'Production Interface Facility - a 412,700\xa0square foot\xa0office building that also includes:\xa0 Laboratory Spaces, Conference Center, Cafeteria , Medical Offices, Computer Laboratory, and Common Space.\nPublic Interface Facility - a 137,157\xa0square foot\xa0office with Laboratory Spaces, Training Offices, 400 Seat Auditorium, Visitor Lobby and Display Space, Central Badging Office, and Common Space.\nThe Turner Logistics team took an initial furniture budget of $13.7 million and reduced it to less than $9 million, without changing any product or the a

In [57]:
df = pd.DataFrame(final_list)
df.to_csv('./turner.csv',index = False)